<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/Test_Jo_brachn/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 48.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import math
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [4]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

Cloning into 'Fallstudie_SCM'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 77 (delta 25), reused 16 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 34.92 KiB | 4.99 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [5]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [6]:
chargers.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [7]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [8]:
dtrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [9]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [10]:
etrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [11]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [12]:
routes

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00
5,s-3,Ditzingen,120,32,09:00,16:00
6,s-4,Ditzingen,120,32,06:30,16:30
7,w1,Ditzingen,100,32,05:30,15:30
8,w2,Ditzingen,100,32,08:00,18:00
9,w3,Ditzingen,100,32,06:45,16:45


### Indexmengen

In [13]:
#Erstellen einer Modellinstanz
scip = Model()

In [21]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(0, 96) # 96 Zeitintervalle pro Tag
I = range(0, 4) # Potenzielle Säulen-Slots
# T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
# print("T_night (Nachtintervalle):", list(T_night))


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
P (potenzielle Fahrzeuge): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
T (Zeitintervalle): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]
I (Säulen): [0, 1, 2, 3]


### Parameter

In [15]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    cons_e[m]    = row["avg_energy_kWh_per_100km"]
    thg_e[m]     = row["thg_yearly"]
    pmax_veh[m]  = row["max_power"]
    batt_kwh[m]  = row["soc_max_kWh"]

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[m] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[m] = row["avg_diesel_per_100km"]

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden
diesel = 1.60

# Netz & Tarif
P_grid_max = 500.0 # max kW am Depot
P_grid_add = 500.0 # Zusatzleistung bei Netzausbau
capex_grid_add = 10000.0 # Kosten/Jahr für Netzausbau

c_energy = 0.25   # Arbeitspreis Strom in €/kWh
c_capex = 1000.0   # Stromkosten in €/a
c_peak = 150.0    # Leistungspreis in €/kW

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_capex_bat_kW = 30.0      # Batteriekosten in €/kW
c_capex_bat_kWh = 350.0     # Batteriekosten in €/kWh
c_opex_bat = 0.02*c_capex_bat_kW*c_capex_bat_kWh # hier muss noch mit der tatsächlichen kW und kWh Anzahl multipliziert werden!!!
roundtrip_eff = 0.98 # Round-Trip Efficiency aka Wirkungsgrad
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975 # Max. Entladetiefe
soc_bat_min_frac = 1.0 - dod    # = 0.025


# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print()

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

Beispiel dist_total[r]: [('t-4', 250), ('t-5', 250), ('t-6', 250)]

Charger pmax (kW): {'Alpitronic-50': 50, 'Alpitronic-200': 200, 'Alpitronic-400': 400}

e-Lkw batt_kwh: {'eActros600': 621, 'eActros400': 414}

Energiebedarf Beispiel (erste Route, e400/e600):
  ('t-4', 'eActros600') = 275.0 kWh
  ('t-4', 'eActros400') = 262.5 kWh

Konstanten: P_grid_max = 500.0 | c_energy = 0.25 | c_toll = 0.34

Speicher: eta = 0.9899494936611666 | soc_bat_min_frac = 0.025000000000000022



In [16]:
def calc_kost_lkw_tour(r,l):

  if (l == 'ActrosL'):  # Ob Diesel oder E
    verbrauch_100km =  cons_d[l]
    kosten_diesel = dist_total[r]/100*verbrauch_100km*diesel
    kosten_maut = c_toll*dist_toll[r]
    return kosten_diesel+kosten_maut
  else:
    kosten_strom = E_route_e[r,l] * c_energy
    return kosten_strom

In [17]:
#Kosten Params LKW Pro Tour

kosten_matrix_lkw = {}

print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)

for r in R:
  for l in L:
    kosten_matrix_lkw[r,l] = calc_kost_lkw_tour(r,l)

print("Kostenpatrix")
print(kosten_matrix_lkw)


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
Kostenpatrix
{('t-4', 'ActrosL'): 155.0, ('t-4', 'eActros600'): 68.75, ('t-4', 'eActros400'): 65.625, ('t-5', 'ActrosL'): 155.0, ('t-5', 'eActros600'): 68.75, ('t-5', 'eActros400'): 65.625, ('t-6', 'ActrosL'): 155.0, ('t-6', 'eActros600'): 68.75, ('t-6', 'eActros400'): 65.625, ('s-1', 'ActrosL'): 60.800000000000004, ('s-1', 'eActros600'): 33.0, ('s-1', 'eActros400'): 31.5, ('s-2', 'ActrosL'): 60.800000000000004, ('s-2', 'eActros600'): 33.0, ('s-2', 'eActros400'): 31.5, ('s-3', 'ActrosL'): 60.800000000000004, ('s-3', 'eActros600'): 33.0, ('s-3', 'eActros400'): 31.5, ('s-4', 'ActrosL'): 60.800000000000004, ('s-4', 'eActros600'): 33.0, ('s-4', 'eActros400'): 31.5, ('w1', 'ActrosL'): 52.480000000000004, ('w1', 'eActros600'): 27.5, ('w1', 'eActros40

### Entscheidungsvariablen

In [18]:
Flottenwahl = {}
for l in L:
    for p in P:
        Flottenwahl[l, p] = scip.addVar(vtype = 'B', name =f"LKW_{l}_auf_ID_{p}")

Zuordnung_LKW_Route = {}
for r in R:
    for p in P:
        Zuordnung_LKW_Route[r, p] = scip.addVar(vtype = 'B', name =f"Route_{r}_wird_von_LKW_mit_ID_{p}_gefahren")

Auswahl_Charger = {}
for c in C:
    for i in I:
        Auswahl_Charger[c, i] = scip.addVar(vtype = 'B', name =f"Charger_{i}_ist_Säule_Nummer_{i}")

Zuordnung_LKW_Zeitpunkt_Charger_Ladepunkt = {}
for p in P:
    for t in T:
        for c in C:
            Zuordnung_LKW_Zeitpunkt_Charger_Ladepunkt = scip.addVar(vtype = 'B', name =f"LKW_{l}_zum_Zeitpunkt_{t}_an_Charger_{c}")

SOC_Zeit_Fahrzeug = {}
Ladeleistung = {}
for p in P:
  for t in T:
    SOC_Zeit_Fahrzeug = scip.addVar(vtype = 'C',lb=0, name=f"SOC_FahrzeugID_{p}_Zeitpunkt{t}")
    Ladeleistung[p, t] = scip.addVar(vtype='C', lb=0, name=f"Ladeleistung_{p}_{t}")




Einsatz = {}
for r in R:
    for l in L:
        for p in P:
            # Diese Variable ist 1, wenn ID p Route r fährt UND Typ l ist
            Einsatz[r, l, p] = scip.addVar(vtype='B', name=f"Einsatz_{r}_{l}_{p}")


Ausbau_Netz = scip.addVar(vtype='B', name="Netzausbau")

Ausbau_Bat = scip.addVar(vtype='B', name="Batterie_kaufen")

Batterie_kWh = scip.addVar(vtype='C', lb=0, name="Batteriekapa_kWh")
Batterie_kW = scip.addVar(vtype='C', lb=0, name="Batterieleistung_kW")

len(scip.getVars())



9356

### Restriktionen

In [19]:
#Für Einsatz{}
for r in R:
    for l in L:
        for p in P:
            # Wenn Einsatz=1, MUSS Zuordnung=1 sein
            scip.addCons(Einsatz[r, l, p] <= Zuordnung_LKW_Route[r, p])
            # Wenn Einsatz=1, MUSS Flottenwahl=1 sein
            scip.addCons(Einsatz[r, l, p] <= Flottenwahl[l, p])
            # Wenn beide 1 sind, MUSS Einsatz 1 werden
            scip.addCons(Einsatz[r, l, p] >= Zuordnung_LKW_Route[r, p] + Flottenwahl[l, p] - 1)

#Für Zuordnung_LKW_Route{}
#1. Jede Route muss genau einmal bedient werden
for r in R:
  scip.addCons(quicksum(Zuordnung_LKW_Route[r,p] for p in P)==1)


# Flottenwahl{}
#1. Max 20 Fahrzeuge
scip.addCons(quicksum(Flottenwahl[l,p] for p in P for l in L)<=20)

#2. Jede ID hat genau 1 LKW Model
for p in P:
  scip.addCons(quicksum(Flottenwahl[l,p] for l in L)==1)


# SOC_Zeit_Fahrzeug{}
#1. Berechnung SOC zum Zeitpunkt t


#2. Regel dass SOC am Anfang des Tages = SOC Ende des Tages sein muss
for p in P:
  scip.addCons(SOC_Zeit_Fahrzeug[p,0] == SOC_Zeit_Fahrzeug[p, 95])

### Zielfunktion

In [20]:
scip.setObjective(quicksum(kosten_matrix_lkw[r, l] * Einsatz[r, l, p] for r in R for l in L for p in P), sense="minimize")

### Berechnung des Ergebnisses